In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px
import pandas as pd

In [3]:
df = pd.read_csv('generated_csvs/2022_2022.csv')
df['work_sustainable_dev_goal'] = df['work_sustainable_dev_goal'].str.replace('-1', 'Uncategorized')
df

,author_id,author_name,author_raw_name,author_department,author_position,author_is_corresponding,author_role,work_id,work_doi,work_display_name,...,work_publication_year,work_publisher,work_journal,work_sustainable_dev_goal,work_topic,work_type.1,work_is_open_access,work_cited_by_count,work_created_date,work_updated_date
0,https://openalex.org/A5018739374,Aaron M. Bauer,Aaron M. Bauer,Herpetology,last,0,Research Associate,https://openalex.org/W4312850930,https://doi.org/10.1007/978-3-031-06153-0_19,The Terrestrial Reptiles of the Gulf of Guinea...,...,2022,Springer Nature,Springer eBooks,Life below water,Conservation and Management of Marine Turtles ...,book-chapter,1,4,2023-01-05,2024-05-21 12:34:09
1,https://openalex.org/A5032036549,Alejandra Hernández‐Agreda,Alejandra Hernandez‐Agreda,Invertebrate Zoology and Geology,first,1,Research Scientist,https://openalex.org/W4226315466,https://doi.org/10.1111/conl.12875,Hidden in the deep: Distinct benthic trajector...,...,2022,Wiley,Conservation letters,Life below water,Resilience of Coral Reef Ecosystems to Climate...,article,1,3,2022-05-05,2024-05-22 10:24:04
2,https://openalex.org/A5032036549,Alejandra Hernández‐Agreda,Alejandra Hernández-Agreda,Invertebrate Zoology and Geology,middle,0,Research Scientist,https://openalex.org/W4283749290,https://doi.org/10.1016/j.scitotenv.2022.157049,Mesophotic depths hide high coral cover commun...,...,2022,Elsevier BV,Science of the total environment,Life below water,Resilience of Coral Reef Ecosystems to Climate...,article,0,3,2022-07-02,2024-05-16 07:33:07
3,https://openalex.org/A5004407993,Alison L. Gould,A. L. Gould,Ichthyology,first,0,Research Scientist,https://openalex.org/W4221123915,https://doi.org/10.1093/gbe/evac044,Chromosome-Level Genome Assembly of the Biolum...,...,2022,Oxford University Press,Genome biology and evolution,Life below water,RNA Sequencing Data Analysis,article,1,3,2022-04-03,2024-05-14 20:43:13
4,https://openalex.org/A5004407993,Alison L. Gould,AL Gould,Ichthyology,first,1,Research Scientist,https://openalex.org/W4304136099,https://doi.org/10.1101/2022.10.10.511632,Highly contiguous genome assemblies of<i>Photo...,...,2022,Cold Spring Harbor Laboratory,bioRxiv (Cold Spring Harbor Laboratory),Life below water,Immunological Responses in Aquatic Organisms,preprint,1,1,2022-10-11,2024-05-11 18:50:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,https://openalex.org/A5077779232,Tomio Iwamoto,"Iwamoto, Tomio",Ichthyology,middle,0,Emeritus,https://openalex.org/W4309889161,https://doi.org/10.11646/zootaxa.5213.2.2,A new species of the grenadier genus Coelorinc...,...,2022,Q15088586,Zootaxa,Life below water,Conservation of Sharks and Rays,article,0,0,2022-11-29,2024-05-18 11:00:13
217,https://openalex.org/A5077779232,Tomio Iwamoto,TOMIO IWAMOTO,Ichthyology,middle,0,Emeritus,https://openalex.org/W4206135947,https://doi.org/10.11646/zootaxa.5092.2.2,Redescription of Ateleopus indicus Alcock 1891...,...,2022,Q15088586,Zootaxa,Life below water,Length-Weight Relationships of Fish Species,article,0,0,2022-01-26,2024-05-21 00:10:03
218,https://openalex.org/A5065650059,Tony Iwane,Tony Iwane,NaN,middle,0,NaN,https://openalex.org/W4308708771,https://doi.org/10.1371/journal.pbio.3001843,The benefits of contributing to the citizen sc...,...,2022,Public Library of Science,PLoS biology,Life on land,Species Distribution Modeling and Climate Chan...,article,1,27,2022-11-14,2024-05-13 02:48:49
219,https://openalex.org/A5043258898,Vanessa Handley,Vanessa Handley,NaN,middle,0,NaN,https://openalex.org/W4286357399,https://doi.org/10.1007/978-3-030-99742-7_23,A New Perspective on Seed Morphological Featur...,...,2022,Springer Nature,Springer eBooks,Zero hunger,Evolution and Classification of Flowering Plants,book-chapter,0,3,2022-07-21,2024-05-11 01:12:43


In [4]:
goal_counts= df.groupby('work_sustainable_dev_goal').size().reset_index(name='counts')
goal_counts = goal_counts.sort_values('counts')
goal_counts['work_sustainable_dev_goal'] = goal_counts['work_sustainable_dev_goal'].str.replace('-1', 'Uncategorized')
goal_counts

,work_sustainable_dev_goal,counts
0,Climate action,1
4,No poverty,1
6,"Peace, justice, and strong institutions",1
7,Quality education,1
8,Sustainable cities and communities,1
5,Partnerships for the goals,3
1,Good health and well-being,5
10,Zero hunger,5
9,Uncategorized,49
3,Life on land,67


In [5]:
app = dash.Dash(__name__)

# Define the components of the layout
app.layout = html.Div([
    dcc.Graph(id='bar-plot', figure=px.bar(goal_counts,
                                           x='work_sustainable_dev_goal',
                                           y='counts')),
    html.H4(id='table-title', style={'textAlign': 'center', 'color': 'blue'}),  # Placeholder for dynamic title
    html.Div(id='details-display')
])

# Callbacks to control what clicking a bar does
@app.callback(
    [Output('table-title', 'children'),
     Output('details-display', 'children')],
    [Input('bar-plot', 'clickData')]
)

def display_click_data(clickData):
    # print("Callback triggered")  # Debug print
    if clickData is not None:
        selected_goal = clickData['points'][0]['x']
        filtered_data = df[df['work_sustainable_dev_goal'] == selected_goal]
        # print(f"Filtered data for {selected_goal}: {filtered_data}")  # Debug print

        title = f'Publications for {selected_goal}'

        table = dash_table.DataTable(
            data=filtered_data.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in filtered_data.columns],
            style_table={'overflowX': 'auto'},
            style_cell={'minWidth': '80px', 'width': '120px', 'maxWidth': '180px', 'textAlign': 'left',
                        'whiteSpace': 'normal',  # Allows text wrapping
                        'height': 'auto' # Adjust cell height to fit content
                        },  
            filter_action='native',
            sort_action='native',
            page_action='native',
            page_size=10,
            export_format='csv',  # Enable export to CSV
            export_headers='display'
        )
        return title, table
    return "Click a bar to see publications for each goal.", None

if __name__ == '__main__':
    app.run_server(debug=True)

# Works by author

In [5]:
df = pd.read_csv('email_attachments/works_by_author_2022.csv')
df['work_sustainable_dev_goal'] = df['work_sustainable_dev_goal'].str.replace('-1', 'Uncategorized')
df

,author_id,author_name,author_raw_name,author_department,author_position,author_is_corresponding,author_role,work_id,work_doi,work_display_name,work_publication_date,work_publication_year,work_publisher,work_journal,work_sustainable_dev_goal,work_is_open_access,work_cited_by_count
0,https://openalex.org/A5018739374,Aaron M. Bauer,Aaron M. Bauer,Herpetology,last,0,Research Associate,https://openalex.org/W4312850930,https://doi.org/10.1007/978-3-031-06153-0_19,The Terrestrial Reptiles of the Gulf of Guinea...,2022-01-01,2022,Springer Nature,Springer eBooks,Life below water,1,3
1,https://openalex.org/A5030612844,Adrian Van Allen,Adrian Van Allen,Anthropology,middle,0,Research Associate,https://openalex.org/W4296816995,https://doi.org/10.1111/muan.12228,Issue Information,2022-09-01,2022,Wiley-Blackwell,Museum anthropology,Uncategorized,1,0
2,https://openalex.org/A5032036549,Alejandra Hernández‐Agreda,Alejandra Hernández-Agreda,Invertebrate Zoology and Geology,middle,0,Research Scientist,https://openalex.org/W4283749290,https://doi.org/10.1016/j.scitotenv.2022.157049,Mesophotic depths hide high coral cover commun...,2022-10-01,2022,Elsevier BV,Science of the total environment,Life below water,0,3
3,https://openalex.org/A5032036549,Alejandra Hernández‐Agreda,Alejandra Hernandez‐Agreda,Invertebrate Zoology and Geology,first,1,Research Scientist,https://openalex.org/W4226315466,https://doi.org/10.1111/conl.12875,Hidden in the deep: Distinct benthic trajector...,2022-02-03,2022,Wiley,Conservation letters,Life below water,1,3
4,https://openalex.org/A5004407993,Alison L. Gould,A L Gould,Ichthyology,first,0,Research Scientist,https://openalex.org/W4221123915,https://doi.org/10.1093/gbe/evac044,Chromosome-Level Genome Assembly of the Biolum...,2022-03-29,2022,Oxford University Press,Genome biology and evolution,Life below water,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,https://openalex.org/A5043258898,Vanessa Handley,Vanessa Handley,NaN,middle,0,NaN,https://openalex.org/W4286357399,https://doi.org/10.1007/978-3-030-99742-7_23,A New Perspective on Seed Morphological Featur...,2022-01-01,2022,Springer Nature,Springer eBooks,Zero hunger,0,3
318,https://openalex.org/A5090548106,W. Brian Simison,W. Brian Simison,CCG,last,0,Curator,https://openalex.org/W4221060004,https://doi.org/10.1186/s12915-022-01271-w,Phylogenomic analysis of Syngnathidae reveals ...,2022-03-27,2022,BioMed Central,BMC biology,Life below water,1,21
319,https://openalex.org/A5090548106,W. Brian Simison,W. Brian Simison,CCG,last,0,Curator,https://openalex.org/W4394314429,https://doi.org/10.6084/m9.figshare.19445186,Additional file 2 of Phylogenomic analysis of ...,2022-01-01,2022,Figshare (United Kingdom),Figshare,Uncategorized,1,0
320,https://openalex.org/A5090548106,W. Brian Simison,W Brian Simison,CCG,middle,0,Curator,https://openalex.org/W4308181751,https://doi.org/10.1111/ele.14135,The gut microbiome reflects ancestry despite d...,2022-11-04,2022,Wiley-Blackwell,Ecology letters,Life on land,0,6


In [7]:
dept_counts= df.groupby('author_department').size().reset_index(name='counts')
dept_counts = dept_counts.sort_values('counts')
# dept_counts['author_department'] = dept_counts['author_department'].str.replace('-1', 'Uncategorized')
dept_counts

,author_department,counts
0,Anthropology,1
11,Icthyology,1
7,Education,1
15,Planetarium,2
16,iNaturalist,2
14,O&M,6
6,Coral Regeneration Lab,7
4,Center for Biodiversity and Community Science,7
1,Aquarium,8
5,Center for Exploration and Travel Health,12


In [9]:
app = dash.Dash(__name__)

# Define the components of the layout
app.layout = html.Div([
    dcc.Graph(id='bar-plot', figure=px.bar(dept_counts,
                                           x='author_department',
                                           y='counts')),
    html.H4(id='table-title', style={'textAlign': 'center', 'color': 'blue'}),  # Placeholder for dynamic title
    html.Div(id='details-display')
])

# Callbacks to control what clicking a bar does
@app.callback(
    [Output('table-title', 'children'),
     Output('details-display', 'children')],
    [Input('bar-plot', 'clickData')]
)

def display_click_data(clickData):
    # print("Callback triggered")  # Debug print
    if clickData is not None:
        selected_dept = clickData['points'][0]['x']
        filtered_data = df[df['author_department'] == selected_dept]
        # print(f"Filtered data for {selected_goal}: {filtered_data}")  # Debug print

        title = f'Publications for {selected_dept}'

        table = dash_table.DataTable(
            data=filtered_data.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in filtered_data.columns],
            style_table={'overflowX': 'auto'},
            style_cell={'minWidth': '80px', 'width': '120px', 'maxWidth': '180px', 'textAlign': 'left',
                        'whiteSpace': 'normal',  # Allows text wrapping
                        'height': 'auto' # Adjust cell height to fit content
                        },  
            filter_action='native',
            sort_action='native',
            page_action='native',
            page_size=10,
            export_format='csv',  # Enable export to CSV
            export_headers='display'
        )
        return title, table
    return "Click a bar to see publications for each department.", None

if __name__ == '__main__':
    app.run_server(debug=True)